# Create a table with SRA IDs, readstats QC data, and submitter metadata

**Below are the steps taken in this notebook:**
1. Installs, import Statements & Global Variable Definitions
2. Read in tables as dataframes
3. Merge dataframes
4. Write merged dataframe to data table and file


## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [3]:
from firecloud import fiss
import pandas as pd      
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [4]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

# table filenames (expected in the workspace bucket, see below)
submitter_metadata = '220118_HPRC_PacBio_HiFi_Metadata_Submission_v0.2_UW_wh_km.txt'
sra_metadata = 'metadata-11130351-processed-ok.tsv'

# submission id
submission_id = 'UW_HPRC_HiFi_Y2'

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_UW_HPRC_HiFi_Y2
Workspace storage bucket: gs://fc-21b07b99-b484-46a3-91e0-9f4bf83ab3aa/


# Read In tables as dataframes

## Readstats table
Find the individual readstats output files in the table and concatenate their contents.
Add the hifi filename and sample name.

In [5]:
# readstats
readstats_df = table_to_dataframe("readstats", workspace=WORKSPACE, workspace_namespace=PROJECT)
qc_list = []

for index, row in readstats_df.iterrows():

        sample_readstats_fp = row['ReadStatsReport']
        sample_readstats_fn = os.path.basename(sample_readstats_fp)
        df = pd.read_csv(sample_readstats_fp, header=None, sep='\t')
        df = df[df[0]=='sample.fastq']
        # add sample and file name
        #df = df.append({0:'undef', 1:'sample', 2:row['sample']}, ignore_index=True)
        df = df.append({0:'undef', 1:'filename', 2:os.path.basename(row['hifi'])}, ignore_index=True)
        # remove all but the variables (total_bp, quartile_25 etc)
        df2 = df.drop(df.columns[[0, 1]], axis=1)
        # make rownames ('total_bp', 'quartile_25' etc)
        df2.index = df.iloc[:,1]
        # remove the now meaningless 'sample.fastq' filename
        df2 = df2.drop(index=['file'])
        qc_list.append(df2)

# merge
readstats_df = pd.concat(qc_list, axis=1).transpose()
readstats_df.head()

1,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,quartile_50,quartile_75,N25,N50,N75,filename
2,1872206,37253071882,37.25,86,58561,19897,16571,19217,22526,17312,20226,23731,m54329U_210524_174516.hifi_reads.bam
2,1434335,27532337755,27.53,86,49254,19195,16014,18446,21636,16660,19398,22815,m64076_210516_091844.hifi_reads.bam
2,2141718,39074294250,39.07,681,55519,18244,15452,17497,20372,15933,18283,21409,m54329U_210718_084331.hifi_reads.bam
2,147915,3026195060,3.03,769,42220,20459,17306,19625,22805,17858,20494,23946,m54329U_210507_224951-bc1011_BAK8A_OA.hifi_rea...
2,1774809,33629205391,33.63,85,54155,18948,16068,18290,21215,16603,19088,22195,m64076_210604_225751.hifi_reads.bam


## Submitter metadata table

In [6]:
meta_fp = os.path.join(bucket, submitter_metadata)
metadata_df = pd.read_csv(meta_fp, sep='\t')
metadata_df = metadata_df[['filename', 'sample_ID', 'instrument_model', 'shear_method', 'size_selection', 'ccs_algorithm', 
    'polymerase_version', 'seq_plate_chemistry_version', 'generator_facility', 'generator_contact']]
metadata_df.head()

,filename,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m54329U_210524_174516.hifi_reads.bam,HG00642,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
1,m64076_210516_091844.hifi_reads.bam,HG00642,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
2,m64076_210520_213355.hifi_reads.bam,HG00642,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
3,m64076_210522_082905.hifi_reads.bam,HG00642,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
4,m54329U_210624_221223.hifi_reads.bam,HG01346,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu


## SRA table
Must download after finishing submission.
The downloaded table needs to be split in one row per file (instead of filename, filename2 <...> filenameN)

In [7]:
## Read in SRA file and split by filename

sra_file = os.path.join(bucket + sra_metadata)
sample_df = pd.read_csv(sra_file, sep='\t')

fnames = [x for x in sample_df.columns if x.startswith('filename')]
sra_df = pd.melt(sample_df, id_vars =['sample_name', 'accession'], value_vars = fnames, value_name='file')
sra_df= sra_df.dropna()
sra_df = sra_df[['file', 'accession', 'sample_name']]
sra_df.columns = ['filename', 'accession', 'sample']
sra_df.head()


,filename,accession,sample
0,m54329U_210524_174516.hifi_reads.bam,SRR18158610,HG00642
1,m54329U_210624_221223.hifi_reads.bam,SRR18158609,HG01346
2,m64076_210526_105450.hifi_reads.bam,SRR18158604,HG01346
3,m54329U_210507_224951-bc1011_BAK8A_OA.hifi_rea...,SRR18158591,HG02071
4,m64076_210604_225751.hifi_reads.bam,SRR18158592,HG02071


# Merge all dataframes

In [8]:
print(sra_df.shape, readstats_df.shape, metadata_df.shape)
# two samples of HG02572 were not uploaded to SRA with the rest of this batch

(80, 3) (82, 13) (82, 10)


In [9]:
merged_df = pd.merge(
    sra_df,
    readstats_df,
    on='filename')
merged_df = pd.merge(
    merged_df,
    metadata_df,
    on='filename')
merged_df.head()

,filename,accession,sample,total_reads,total_bp,total_Gbp,min,max,mean,quartile_25,...,N75,sample_ID,instrument_model,shear_method,size_selection,ccs_algorithm,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact
0,m54329U_210524_174516.hifi_reads.bam,SRR18158610,HG00642,1872206,37253071882,37.25,86,58561,19897,16571,...,23731,HG00642,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
1,m54329U_210624_221223.hifi_reads.bam,SRR18158609,HG01346,1971103,36295801919,36.30,86,58600,18413,15656,...,21555,HG01346,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
2,m64076_210526_105450.hifi_reads.bam,SRR18158604,HG01346,1059488,19327631225,19.33,85,40848,18242,15827,...,20196,HG01346,PacBio Sequel II,Megaruptor 3,SageELF,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
3,m54329U_210507_224951-bc1011_BAK8A_OA.hifi_rea...,SRR18158591,HG02071,147915,3026195060,3.03,769,42220,20459,17306,...,23946,HG02071,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu
4,m64076_210604_225751.hifi_reads.bam,SRR18158592,HG02071,1774809,33629205391,33.63,85,54155,18948,16068,...,22195,HG02071,PacBio Sequel II,Megaruptor 3,PippinHT,6.0.0,P2.2,C2,University of Washington,kmiyamot@uw.edu


# Upload To Tables

In [10]:
## Create table
#dataframe_to_table("post_sra_metadata", merged_df, WORKSPACE, PROJECT)
outf = os.path.join(bucket, submission_id + '_post_sra_metadata.tsv')
merged_df.to_csv(outf, sep="\t", index=False)

In [11]:
merged_df.shape

(80, 24)